# RAG Example: Build a chatbot based on an investment paper

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [116]:
import util
import os
import openai
from openai import OpenAI
import json
import numpy as np
import panel as pn
pn.extension()

openai.api_key = util.get_openai_api_key()
os.environ["OPENAI_API_KEY"] = openai.api_key

In [67]:
client = OpenAI(api_key=openai.api_key)
GPT3dot5_MODEL = 'gpt-3.5-turbo'
GPT4O_MODEL = 'gpt-4o-mini'

def get_completion_from_messages(messages, model=GPT3dot5_MODEL, temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

In [69]:
### generate some relevant evaluation questions

delimiter = "####"
system_message = f"""
You are an expert on economics and investment field. \
You will be asked to provide insights on a paper on factors affecting GDP growth. \
The query will be delimited with \
{delimiter} characters.
You will provide answers based on the paper. \

"""
user_message = f"""\
Can you identify 5 key questions that the paper "Demographics and Productivity: Drivers of Economic Growth" may have studied?
"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

In [71]:
print(messages)

[{'role': 'system', 'content': '\nYou are an expert on economics and investment field. You will be asked to provide insights on a paper on factors affecting GDP growth. The query will be delimited with #### characters.\nYou will provide answers based on the paper. \n'}, {'role': 'user', 'content': '####Can you identify 5 key questions that the paper "Demographics and Productivity: Drivers of Economic Growth" may have studied?\n####'}]


In [73]:
response = get_completion_from_messages(messages)
print(response)

1. How does the age distribution of the population impact economic growth?
2. What is the relationship between workforce productivity and GDP growth?
3. How do changes in fertility rates affect economic growth?
4. What role do immigration patterns play in shaping GDP growth?
5. How do advancements in technology and automation impact productivity and economic growth?


In [75]:
question_list = response.split('\n')
print(question_list)

['1. How does the age distribution of the population impact economic growth?', '2. What is the relationship between workforce productivity and GDP growth?', '3. How do changes in fertility rates affect economic growth?', '4. What role do immigration patterns play in shaping GDP growth?', '5. How do advancements in technology and automation impact productivity and economic growth?']


In [77]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./demographics-productivity-drivers-econgrowth.pdf"]
).load_data()

In [79]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

37 

<class 'llama_index.core.schema.Document'>
Doc ID: 27f800db-51c1-4df7-87b8-93b41d6c6826
Text: Demographics and Productivity:  Drivers of Economic Growth
NOVEMBER | 2023


In [81]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

## Building query engine based on RAG using window-sentence retrieval

In [84]:
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage
from llama_index.core import Settings

def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    Settings.llm=llm
    Settings.embed_model=embed_model
    Settings.node_parser=node_parser
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [86]:
from llama_index.llms.openai import OpenAI

index = build_sentence_window_index(
    [document],
    llm=OpenAI(model=GPT3dot5_MODEL, temperature=0.0),
    save_dir="./sentence_index",
)

In [88]:
query_engine = get_sentence_window_query_engine(index, similarity_top_k=6)

In [90]:
user_input = question_list[0]
response = query_engine.query(user_input)
print(response)

Higher population levels, especially in the primary working ages of 15-64, generate natural GDP growth even when productivity is stagnant.


### Building a simple chatbot interface

In [93]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [95]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    response = query_engine.query(user_input).response
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    print(response)
    return pn.Column(*panels)

In [97]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Knowledge Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Knowledge Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)

### Evaluation of chatbot using TruLens

In [100]:
eval_questions = [
    'How do demographic changes, such as aging populations or shifts in birth rates, impact overall GDP growth rates in different countries?',
    'What role does labor force participation play in influencing productivity and economic growth across various demographic groups?',
    'How do educational attainment and skill levels within a population affect productivity and, consequently, GDP growth?',
    'In what ways do immigration patterns contribute to or detract from the productivity and economic growth of a nation?',
    'How do technological advancements interact with demographic factors to influence productivity and economic growth trajectories?'    
]

In [ ]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [110]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        print(question)
        with tru_recorder as recording:
            response = query_engine.query(question)
            print(response)

from trulens_eval import (
Feedback,
TruLlama,
    )
from trulens_eval.feedback import GroundTruthAgreement
from trulens_eval.feedback.provider.openai import OpenAI as TruOpenAI

def get_prebuilt_trulens_recorder(query_engine, app_id):

    openai = TruOpenAI()

    qa_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
        .on_input_output()
    )

    qs_relevance = (
        Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
        .on_input()
        .on(TruLlama.select_source_nodes().node.text)
        .aggregate(np.mean)
    )

#    grounded = TruOpenAI() #or another feedback provider of your choice
#    groundedness = (
#        Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
#        .on(Select.RecordCalls.retrieve.rets.collect())
#        .on_output()
#        .aggregate(grounded.grounded_statements_aggregator)
#    )
    
    feedbacks = [qa_relevance, qs_relevance]#, groundedness]
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
    )
    return tru_recorder

In [104]:
Tru().reset_database()

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [118]:
tru_recorder = get_prebuilt_trulens_recorder(
    query_engine,
    app_id='sample_query_engine'
)

run_evals(eval_questions, tru_recorder, query_engine)

Tru().run_dashboard()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.sch

Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.15:52093 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>